#### Using column_property for column level options

_Options_ can be specified when mapping a `Column` using the `column_property()` function. This function __explicitly creates the `ColumnProperty`__ used by the `mapper()` to __keep track of the `Column`__; normally, the `mapper()` creates this __automatically__. Using `column_property()`, we can _pass additional arguments_ about how we'd like the `Column` to be mapped. Below, we pass an option *active_history*, which specifies that a change to this column's value should result in the former value being loaded first.

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String, Table, MetaData
from sqlalchemy.orm import declarative_base, column_property

In [2]:
metadata_obj = MetaData()
Base = declarative_base()

In [3]:
class BaseUser(Base):
    __tablename__ = "base_user"
    
    id = Column(Integer, primary_key=True)
    name = column_property(Column(String(50)), active_history=True)

`column_property()` is also used to __map a single attribute to multiple columns__. This use case arises when _mapping to_ a `join()` which has attributes which are equated to each other.

In [4]:
user_table = Table(
    "user",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

In [5]:
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", Integer, ForeignKey("user.id")),
    Column("email_address", String),
)

In [6]:
class User(Base):
    __table__ = user_table.join(address_table)
    
    # assign "user.id", "address.user_id" to the "id" attribute
    id = column_property(user_table.c.id, address_table.c.user_id)
    address_id = address_table.c.id

Another place where `column_property()` is needed is to __specify SQL expressions as mapped attributes__, such as below where we _create an attribute_ fullname that is the _string concatenation_ of the `firstname` and `lastname` columns.

In [7]:
class ConcatUser(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    firstname = Column(String(50))
    lastname = Column(String(50))
    fullname = column_property(firstname + " " + lastname)